In [209]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import Select

chromedriver_autoinstaller.install()
options = Options()
driver = webdriver.Chrome('./chromedriver', chrome_options=options)

from utils import * 
driver.get("https://mcols.autoever.com/secure/Dashboard.jspa")

ID = "10896665"
PW = "A12345678!"

enter_to_mcols(driver, ID, PW)


C:\Users\HAEA_MI\AppData\Local\Temp\ipykernel_3492\2077066791.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)
C:\Users\HAEA_MI\AppData\Local\Temp\ipykernel_3492\2077066791.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)


In [258]:

def type_retriever(driver, issue_type = 'Bug') :

    if issue_type == 'Bug' :
        tab = driver.find_element(By.ID, "tab-0")
    else :
        tab = driver.find_element(By.CLASS_NAME, "content")
    children = tab.find_elements(By.XPATH, "*")

    retval = {}

    for child in children :

        # initialize inputs
        field_name = None
        input_type = ""
        required = False

        # check input element is required or not
        if  len(child.find_elements(By.CLASS_NAME, 'icon-required')) > 0 :
            required = True
        else :
            required = False
        
        child_class = child.get_attribute('class')
        if 'field-group' in child_class :
            # find field name
            input_type += 'field-group'
            field_name = child.find_element(By.TAG_NAME, 'label').get_attribute("innerText").split('\n')[0]

            dynamicselects = child.find_elements(By.CLASS_NAME, 'dynamic-select-mark')
            jiramultiselects = child.find_elements(By.CLASS_NAME, 'jira-multi-select')
            cfselects = child.find_elements(By.CLASS_NAME, 'cf-select')
            auissselects  = child.find_elements(By.CLASS_NAME, 'aui-ss-select')
            textareas = child.find_elements(By.CLASS_NAME, 'textarea')
            selects = child.find_elements(By.CLASS_NAME, 'select')
            texts = child.find_elements(By.CLASS_NAME, 'text')
            
            if 'aui-field-cascadingselect' in child_class :
                input_type += " aui-field-cascadingselect"

                retval[field_name] = {
                    'input_type' : input_type+" cascadingselect-parent", 
                    'required' : required,
                    'element' : child.find_element(By.CLASS_NAME, 'cascadingselect-parent')
                }

                field_name+='_sub'
                input_type += " cascadingselect-child" 
                child = child.find_element(By.CLASS_NAME, 'cascadingselect-child')

            elif 'aui-field-labelpicker'in child_class :
                input_type += " aui-field-labelpicker"
            elif 'aui-field-datepicker' in child_class :
                input_type += " aui-field-datepicker"

            elif len(auissselects) == 1 :
                input_type += " aui-ss-select"
            elif len(dynamicselects) == 1 :
                input_type += " dynamic-select-mark"
            elif len(jiramultiselects) == 1 :
                input_type += " jira-multi-select"
            elif len(cfselects) == 1 :
                input_type += " cf-select"
            elif len(textareas) == 1 :
                input_type += " textarea"
            elif len(selects) == 1 :
                input_type += " select"
            elif len(texts) == 1 :
                input_type += " text"

        elif 'fieldset' in child.tag_name :
            # find field name 
            input_type += 'fieldset'
            field_name = child.find_element(By.TAG_NAME, 'legend').get_attribute("innerText").split('\n')[0]
            checkbox_elem = child.find_elements(By.TAG_NAME, 'label')
            file_input_elem = child.find_elements(By.CLASS_NAME, 'file-input-list')
            if len(checkbox_elem) > 0 :
                input_type += ' checkbox' 
            
            elif len(file_input_elem) > 0 :
                input_type += ' file-input-list' 
            
            
        elif 'hidden' in child_class:
            input_type += 'hidden'
        else :
            input_type = None

        retval[field_name] = {
            'input_type' : input_type, 
            'required' : required,
            'element' : child
        }
    return retval

In [336]:
import pandas as pd
import numpy as np
press_create_btn(driver)
field_infos = type_retriever(driver, issue_type = 'Bug')
xlsx_file = 'HMC OEM 내비게이션 개발 협업 (HMCOEM)_03-09-2023-13-29-22.xlsx'
field_infos = type_retriever(driver, issue_type = 'Bug')
df = pd.read_excel(xlsx_file, header=0, index_col=0)
df = df.replace({np.nan:None})
data = list(df[0:100].to_dict('series').values())


In [338]:
for data_item in data[:1] :
    for key in data_item.to_dict().keys() :
        if not key in ['Project', 'Issue Type'] :
            if key in list(field_infos.keys()):
                element = field_infos[key]['element']
                input_content = data_item[key]
                input_type = field_infos[key]['input_type']
                if input_content :
                    try :
                        match input_type :
                            case 'field-group dynamic-select-mark' :
                                element = element.find_element(By.TAG_NAME, 'select')
                                Select(element).select_by_visible_text(input_content)                        
                            case 'field-group aui-field-cascadingselect cascadingselect-parent' :
                                Select(element).select_by_visible_text(input_content)
                            case 'field-group aui-field-cascadingselect cascadingselect-child' :
                                Select(element).select_by_visible_text(input_content)
                            case 'field-group aui-field-datepicker' :
                                element = element.find_element(By.TAG_NAME, 'input')
                                element.click()
                                element.clear()
                                element.send_keys(Keys.BACK_SPACE)
                                element.send_keys(input_content)
                            case 'field-group cf-select' :
                                # print(key)
                                element = element.find_element(By.TAG_NAME, 'select')
                                Select(element).select_by_visible_text(input_content)
                            case 'field-group select' :
                                select_elements = element.find_elements(By.TAG_NAME, 'select')
                                input_elements = element.find_elements(By.TAG_NAME, 'input')
                                if len(input_elements) == 1 :
                                    element = input_elements[0]                                
                                    element.click()
                                    element.clear()
                                    element.send_keys(Keys.BACK_SPACE)
                                    element.send_keys(input_content)
                                    element.send_keys(Keys.ENTER)
                                elif len(select_elements) == 1 :
                                    element = select_elements[0]
                                    Select(element).select_by_visible_text(input_content)
                            case 'field-group jira-multi-select' :
                                element = element.find_element(By.TAG_NAME, 'textarea')
                                for item in input_content.split(', ') :
                                    element.send_keys(item)
                                    element.send_keys(Keys.ENTER)
                            case 'field-group text' :
                                element = element.find_element(By.TAG_NAME, 'input')
                                if 'ajs-dirty-warning-exempt' in element.get_attribute('class') :
                                    element.click()
                                    element.send_keys(Keys.BACK_SPACE)
                                    element.clear()
                                    element.send_keys(input_content)
                                    time.sleep(3)
                                    element.send_keys(Keys.ENTER)
                                else :
                                    element.clear()
                                    element.send_keys(input_content)
                            case 'fieldset checkbox' :
                                field = element.find_elements(By.TAG_NAME, 'div')
                                options = list(map(lambda x : x.text, field))
                                checkbox = field[options.index(input_content)].find_element(By.TAG_NAME, 'label')
                                checkbox.click()
                            case 'field-group textarea' :
                                if key == 'Description' :
                                    input_content = input_content.replace('[로그:  ]', '[로그: {} ]'.format(data_item['log']))
                                element = element.find_element(By.CLASS_NAME, 'textarea')
                                element.click()
                                element.send_keys(Keys.BACK_SPACE)
                                element.clear()
                                element.send_keys(input_content)
                                time.sleep(3)
                                element.send_keys(Keys.ENTER)                                                    
                            case 'field-group aui-field-labelpicker' :
                                element = element.find_element(By.TAG_NAME, 'textarea')
                                for item in input_content.split(', ') :
                                    element.send_keys(item)
                                    element.send_keys(Keys.ENTER)
                                pass
                            case 'fieldset file-input-list' :
                                filepath = 'C:\\Users\\HAEA_MI\\Downloads\\Feb13JXIssueList.txt'
                                filepath = 
                                element = element.find_element(By.CLASS_NAME, 'issue-drop-zone__file')
                                element.send_keys(filepath)
                                pass 
                            case 'hidden':
                                pass
                    except AttributeError as e :
                        print('{} field does not support input {}'.format(key, input_content)) 

Vehicle Code JW
Labels 북미사무소
Additional Labels sdfsdfsdfsdf
OEM 배포관리 NO sdfsdfsdfsdf
품질 분류 sdfsdfdsfdsf
Epic/Theme sdfsdfsdf
HAE사양담당 sdfsdfsdf
사양설명회 sdfsdfsdf
사양변경이력 dsfdsfsdf


In [167]:
field_infos

{'Summary': {'input_type': 'field-group text',
  'required': True,
  'element': <selenium.webdriver.remote.webelement.WebElement (session="a15db0516712066557e8127d48a1fce5", element="695d0334-4a35-4ab1-8a54-92bcdb9c299e")>},
 'Security Level': {'input_type': 'field-group select',
  'required': True,
  'element': <selenium.webdriver.remote.webelement.WebElement (session="a15db0516712066557e8127d48a1fce5", element="263a6226-5c51-4ba4-bad4-62b3ba61c021")>},
 'Priority': {'input_type': 'field-group select',
  'required': False,
  'element': <selenium.webdriver.remote.webelement.WebElement (session="a15db0516712066557e8127d48a1fce5", element="230fac98-1ce1-4761-b73c-7fb762f7e73c")>},
 'From': {'input_type': 'field-group dynamic-select-mark',
  'required': False,
  'element': <selenium.webdriver.remote.webelement.WebElement (session="a15db0516712066557e8127d48a1fce5", element="a898dea2-90b6-4ce4-9333-2060e5711545")>},
 'ESTRACK': {'input_type': 'field-group aui-field-labelpicker',
  'require

In [152]:
import os

issue_type = 'Bug'

if issue_type == 'Bug' :
    tab = driver.find_element(By.ID, "tab-0")
else :
    tab = driver.find_element(By.CLASS_NAME, "content")
children = tab.find_elements(By.XPATH, "*")


retval = []

for idx_child, child in enumerate(children) :

    # initialize inputs
    field_name = None
    input_type = ""
    xpath = None
    required = False

    # check input element is required or not
    if  len(child.find_elements(By.CLASS_NAME, 'icon-required')) > 0 :
        required = True
    else :
        required = False
    
    child_class = child.get_attribute('class')
    if 'field-group' in child_class :
        # find field name
        input_type += 'field-group'
        field_name = child.find_element(By.TAG_NAME, 'label').get_attribute("innerText").split('\n')[0]

        cascadingselects = child.find_elements(By.CLASS_NAME, 'aui-field-cascadingselect')
        dynamicselects = child.find_elements(By.CLASS_NAME, 'dynamic-select-mark')
        jiramultiselects = child.find_elements(By.CLASS_NAME, 'jira-multi-select')
        cfselects = child.find_elements(By.CLASS_NAME, 'cf-select')
        auifieldlabelpickers = child.find_elements(By.CLASS_NAME, 'aui-field-labelpicker')
        auissselects  = child.find_elements(By.CLASS_NAME, 'aui-ss-select')
        textareas = child.find_elements(By.CLASS_NAME, 'textarea')
        selects = child.find_elements(By.CLASS_NAME, 'select')
        texts = child.find_elements(By.CLASS_NAME, 'text')
        
        if 'aui-field-cascadingselect' in child_class :
            input_type += " aui-field-cascadingselect"
        elif 'aui-field-labelpicker'in child_class :
            input_type += " aui-field-labelpicker"
        elif 'aui-field-datepicker' in child_class :
            input_type += " aui-field-datepicker"

        elif len(auissselects) == 1 :
            input_type += " aui-ss-select"
        elif len(dynamicselects) == 1 :
            input_type += " dynamic-select-mark"
        elif len(jiramultiselects) == 1 :
            input_type += " jira-multi-select"
        elif len(cfselects) == 1 :
            input_type += " cf-select"
        elif len(textareas) == 1 :
            input_type += " textarea"
        elif len(selects) == 1 :
            input_type += " select"
        elif len(texts) == 1 :
            input_type += " text"

    elif 'fieldset' in child.tag_name :
        # find field name 
        input_type += 'fieldset'
        field_name = child.find_element(By.TAG_NAME, 'legend').get_attribute("innerText").split('\n')[0]
        checkbox_elem = child.find_elements(By.TAG_NAME, 'label')
        file_input_elem = child.find_elements(By.CLASS_NAME, 'file-input-list')
        if len(checkbox_elem) > 0 :
            input_type += ' checkbox' 
        
        elif len(file_input_elem) > 0 :
            input_type += ' file-input-list' 
        
        
    elif 'hidden' in child_class:
        input_type += 'hidden'
    else :
        input_type = None

    retval.append(
        {
            'field_name' : field_name, 
            'input_type' : input_type, 
            'required' : required
        }
    )

In [164]:
for i in retval :
    print(i)

{'field_name': 'Summary', 'input_type': 'field-group text', 'required': True}
{'field_name': 'Security Level', 'input_type': 'field-group select', 'required': True}
{'field_name': 'Priority', 'input_type': 'field-group select', 'required': False}
{'field_name': 'From', 'input_type': 'field-group dynamic-select-mark', 'required': False}
{'field_name': 'ESTRACK', 'input_type': 'field-group aui-field-labelpicker', 'required': False}
{'field_name': 'Product Type', 'input_type': 'field-group aui-field-cascadingselect', 'required': True}
{'field_name': 'Test Method(M)', 'input_type': 'field-group dynamic-select-mark', 'required': True}
{'field_name': 'Problem Type(H)', 'input_type': 'field-group dynamic-select-mark', 'required': True}
{'field_name': 'Problem Type(M)', 'input_type': 'field-group aui-field-cascadingselect', 'required': True}
{'field_name': 'Operation Type(H)', 'input_type': 'field-group dynamic-select-mark', 'required': False}
{'field_name': 'Operation Type(M)', 'input_type': 

In [154]:
set(list(map(lambda x : x['input_type'], retval)))

{'field-group aui-field-cascadingselect',
 'field-group aui-field-datepicker',
 'field-group aui-field-labelpicker',
 'field-group cf-select',
 'field-group dynamic-select-mark',
 'field-group jira-multi-select',
 'field-group select',
 'field-group text',
 'field-group textarea',
 'fieldset checkbox',
 'fieldset file-input-list',
 'hidden'}

In [155]:
for x in retval :
    match x['input_type'] :
        case 'field-group aui-field-cascadingselect' :
            pass
        case 'field-group aui-field-datepicker' :
            pass
        case 'field-group aui-field-labelpicker' :
            pass
        case 'field-group cf-select' :
            pass
        case 'field-group dynamic-select-mark' :
            pass
        case 'field-group jira-multi-select' :
            pass
        case 'field-group select' :
            pass
        case 'field-group text' :
            pass
        case 'field-group textarea' :
            pass
        case 'fieldset checkbox' :
            pass
        case 'fieldset file-input-list' :
            pass 
        case 'hidden':
            pass

In [96]:
import pandas as pd
df = pd.read_excel(xlsx_file, header=0, index_col=0)
data = list(df[0:100].to_dict('series').values())

In [98]:
data_item = data[0]

for key in data_item.to_dict().keys() :
    if not key in ['Project', 'Issue Type'] :
        # fileds_dict = get_project_input_fields_json(driver, 'HMC OEM 내비게이션 개발 협업 (HMCOEM)', 'Bug', 10)
        if key in list(fileds_dict.keys()):
            if fileds_dict[key]['type'] == 'select' :
                try :
                    time.sleep(1)
                    project = fileds_dict[key]['xpath']
                    project.click()
                    # project.clear()
                    # project.send_keys(Keys.BACK_SPACE)
                    project.send_keys(data_item[key])
                    project.send_keys(Keys.ENTER)
                except Exception as e :
                    print(key, e)

Priority Message: element not interactable
  (Session info: chrome=110.0.5481.180)
Stacktrace:
Backtrace:
	(No symbol) [0x00DB6643]
	(No symbol) [0x00D4BE21]
	(No symbol) [0x00C4D960]
	(No symbol) [0x00C823A7]
	(No symbol) [0x00C76B13]
	(No symbol) [0x00C9FD7C]
	(No symbol) [0x00C7641F]
	(No symbol) [0x00CA00D4]
	(No symbol) [0x00CB6B09]
	(No symbol) [0x00C9FB76]
	(No symbol) [0x00C749C1]
	(No symbol) [0x00C75E5D]
	GetHandleVerifier [0x0102A142+2497106]
	GetHandleVerifier [0x010585D3+2686691]
	GetHandleVerifier [0x0105BB9C+2700460]
	GetHandleVerifier [0x00E63B10+635936]
	(No symbol) [0x00D54A1F]
	(No symbol) [0x00D5A418]
	(No symbol) [0x00D5A505]
	(No symbol) [0x00D6508B]
	BaseThreadInitThunk [0x764700F9+25]
	RtlGetAppContainerNamedObjectPath [0x76F17BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76F17B8E+238]

Assignee Message: element not interactable
  (Session info: chrome=110.0.5481.180)
Stacktrace:
Backtrace:
	(No symbol) [0x00DB6643]
	(No symbol) [0x00D4BE21]
	(No symbol) [0x00C

In [82]:
key

'Security Level'

In [76]:
data_item = data[0]

for i in data_item.to_dict().keys() :

    if i == 'Project' :
        project_field = driver.find_element(By.CLASS_NAME, 'issue-setup-fields').find_element(By.ID, 'project-field')
        project_field.click()
        project_field.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '')
        project_field.send_keys(data_item[i])
        project_field.send_keys(Keys.ENTER)
        time.sleep(5)
    elif i == 'Issue Type' :
        issuetype_field = driver.find_element(By.CLASS_NAME, 'issue-setup-fields').find_element(By.ID, 'issuetype-field')
        issuetype_field.click()
        issuetype_field.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '')
        issuetype_field.send_keys(data_item[i])
        issuetype_field.send_keys(Keys.ENTER)
        time.sleep(5)
    else :
        try :
            if fileds_dict[i]['type'] == 'select' :
                fileds_dict[i]['xpath'].click()


            print(fileds_dict[i])
        except Exception as e :
            print(e)

{'type': 'input', 'xpath': <selenium.webdriver.remote.webelement.WebElement (session="a15db0516712066557e8127d48a1fce5", element="0f3b6ac8-a88b-4807-97c2-3018613e8d61")>, 'options': '', 'required': True}
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=110.0.5481.180)
Stacktrace:
Backtrace:
	(No symbol) [0x00DB6643]
	(No symbol) [0x00D4BE21]
	(No symbol) [0x00C4DA9D]
	(No symbol) [0x00C509E4]
	(No symbol) [0x00C508AD]
	(No symbol) [0x00C50B30]
	(No symbol) [0x00C83150]
	(No symbol) [0x00C769AD]
	(No symbol) [0x00C9FD7C]
	(No symbol) [0x00C7641F]
	(No symbol) [0x00CA00D4]
	(No symbol) [0x00CB6B09]
	(No symbol) [0x00C9FB76]
	(No symbol) [0x00C749C1]
	(No symbol) [0x00C75E5D]
	GetHandleVerifier [0x0102A142+2497106]
	GetHandleVerifier [0x010585D3+2686691]
	GetHandleVerifier [0x0105BB9C+2700460]
	GetHandleVerifier [0x00E63B10+635936]
	(No symbol) [0x00D54A1F]
	(No symbol) [0x00D5A418]
	(No symbol) [0x00D5A505]
	(No symbol) [0x00D6508B]
	

In [73]:
set(list(map(lambda x: x[1]['type'], fileds_dict.items())))

{None,
 'cascading select',
 'date',
 'fieldset checkbox',
 'fieldset file-input-list',
 'input',
 'issue-link-edit',
 'labelpicker',
 'rendering select',
 'select',
 'textarea'}

In [84]:
data_item = data[0]
for key in data_item.to_dict().keys() :
    fileds_dict = get_project_input_fields_json(driver, 'HMC OEM 내비게이션 개발 협업 (HMCOEM)', 'Bug', 10)
    if not key in ['Project', 'Issue Type'] :
        if key in list(fileds_dict.keys()):
            if fileds_dict[key]['type'] == 'select' :
                fileds_dict[key]['xpath'].find_element(By.TAG_NAME, 'select')
                time.sleep(1)
                project = driver.find_element(By.XPATH, xpath_priority)
                project.click()
                project.clear()
                project.send_keys(Keys.BACK_SPACE)
                project.send_keys(data_item[key])
                project.send_keys(Keys.ENTER)
                print(fileds_dict[key])


Summary
Security Level
Priority
From
ESTRACK
Product Type
Test Method(M)
Problem Type(H)
Problem Type(M)
Operation Type(H)
Operation Type(M)
Due Date
Component/s
Affects Version/s
Fix Version/s
Assignee
Multi Assignee
Vehicle Code
Environment
Reproducibility
Description
Original Estimate
Remaining Estimate
Labels
Additional Labels
TeMS ID
Baseline start date
Baseline end date
Progress
Linked Issues
OEM 배포관리 NO
품질 분류
Vehicle_Code_Std
Epic Link
Region(Multi Select)
Epic/Theme
Difficulty
HAE사양담당
사양설명회
사양변경이력
Summary
Security Level
Priority
From
ESTRACK
Product Type
Test Method(M)
Problem Type(H)
Problem Type(M)
Operation Type(H)
Operation Type(M)
Due Date
Component/s
Affects Version/s
Fix Version/s
Assignee
Multi Assignee
Vehicle Code
Environment
Reproducibility
Description
Original Estimate
Remaining Estimate
Labels
Additional Labels
TeMS ID
Baseline start date
Baseline end date
Progress
Linked Issues
OEM 배포관리 NO
품질 분류
Vehicle_Code_Std
Epic Link
Region(Multi Select)
Epic/Theme
Difficulty

KeyboardInterrupt: 